In [ ]:

        #base_model = MobileNetV2(input_shape=(224,224,3),include_top=False, weights='imagenet')
        preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
        global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
        base_model.trainable = False
        #net =base_model.output
        inputs = tf.keras.Input(shape=(224, 224, 3))
        #net= data_augmentation(inputs)
        net = preprocess_input(inputs)
        net = base_model(net, training=False)
        net = global_average_layer(net)
        net = tf.keras.layers.Dropout(0.2)(net)
        net = tf.keras.layers.Dense(4096, activation='relu', kernel_regularizer=l2(1e-2), kernel_initializer='he_uniform')(net)
        net = tf.keras.layers.Dense(embeddingsize, activation=None, kernel_regularizer=l2(1e-2), kernel_initializer='he_uniform')(net)
        #Force the encoding to live on the d-dimentional hypershpere
        net = tf.keras.layers.Lambda(lambda x: K.l2_normalize(x,axis=-1))(net)
        base_model =tf.keras.Model(inputs, net, name='tuned_model')

        #base_model.trainable = True
        # Let's take a look to see how many layers are in the base model
        print("Number of layers in the base model: ", len(base_model.layers))
        len(base_model.trainable_variables)

        # Let's take a look to see how many layers are in the base model
        # Fine-tune from this layer onwards
        #fine_tune_at = 100
        # Freeze all the layers before the `fine_tune_at` layer
        #for layer in base_model.layers[:fine_tune_at]:
         #   layer.trainable =  False
             
        #base_model =tf.keras.Model(resnet_model.input, output, name='tuned_model')

In [ ]:
# Allow memory growth for the GPU
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
#def l2Norm(x):
 #       return  K.l2_normalize(x, axis=-1)
#def triplet_loss(y_true, y_pred):
 #   margin = K.constant(0.02)
  #  return K.mean(K.maximum(K.constant(0), K.square(y_pred[:,0,0]) - K.square(y_pred[:,1,0]) + margin))
#def accuracy(y_true, y_pred):
 #   return K.mean(y_pred[:,0,0] < y_pred[:,1,0])
#def euclidean_distance(vects):
 #   x, y = vects
  #  return K.sqrt(K.maximum(K.sum(K.square(x - y), axis=1, keepdims=True), K.epsilon()))

In [ ]:
        # add a global spatial average pooling layer
        #x = base_model.output
        #x = GlobalAveragePooling2D()(x)
        # let's add a fully-connected layer
        #x = Dense(1024, activation='relu')(x)
        # and a logistic layer -- let's say we have 200 classes
        #predictions = Dense(200, activation='softmax')(x)
        # this is the model we will train
        #model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np
base_model = VGG16(input_shape=input_shape,weights='imagenet', include_top=False)
net =base_model.output
net = tf.keras.layers.Dropout(0.5)(net)
net = tf.keras.layers.Flatten(name='new_flatten')(net) 
net = tf.keras.layers.Dense(4096, activation='relu', kernel_regularizer=l2(1e-3), 
                            kernel_initializer='he_uniform',name='new_fc')(net)
net = tf.keras.layers.Dense(embeddingsize, activation=None, kernel_regularizer=l2(1e-3), 
                            kernel_initializer='he_uniform', name='new_fc2')(net)
#Force the encoding to live on the d-dimentional hypershpere
net = tf.keras.layers.Lambda(lambda x: K.l2_normalize(x,axis=-1),name='lamda')(net)
base_model =tf.keras.Model(base_model.input, net, name='tuned_model')
#base_model.trainable = False
# Let's take a look to see how many layers are in the base model
# Fine-tune from this layer onwards
fine_tune_at = 18
# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False
    

In [ ]:
    elif(name=="mobileNet"):
        # Create the base model from the pre-trained model MobileNet V2
        from tensorflow.keras.applications import MobileNetV2
        base_model = tf.keras.applications.MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')
        embeddingsize=50
        #net =base_model.output
        net= base_model.layers[-1].output 
        net = tf.keras.layers.GlobalAveragePooling2D()(net) 
        #net = tf.keras.layers.Dense(1024, activation='relu', kernel_regularizer=l2(1e-3), 
                                    #kernel_initializer='he_uniform')(net)
        net = tf.keras.layers.Dense(embeddingsize, activation=None, kernel_regularizer=l2(1e-3), 
                                    kernel_initializer='he_uniform')(net)
        net = tf.keras.layers.Dropout(0.2)(net)
        #Force the encoding to live on the d-dimentional hypershpere
        net = tf.keras.layers.Lambda(lambda x: K.l2_normalize(x,axis=-1))(net)
        base_model =tf.keras.Model(base_model.input, net, name='tuned_model')
        base_model.trainable = True
        # Let's take a look to see how many layers are in the base model
        # Fine-tune from this layer onwards
        fine_tune_at = 100
        # Freeze all the layers before the `fine_tune_at` layer
        for layer in base_model.layers[:fine_tune_at]:
                layer.trainable =  False

In [ ]:
import matplotlib.pyplot as plt
import skimage.io
classes = load_classes(networkConfig.data_location, networkConfig.extend, networkConfig.min_samples)
train_dir = os.path.join(networkConfig.data_location)
k=1
for i, cl in enumerate(classes):
        dir = os.path.join(train_dir, cl)
        files = [f for f in os.listdir(dir) if f.endswith(networkConfig.extend)]
        for file in files:
            # stopping condition
            image = skimage.io.imread(os.path.join(dir, file))
            fig=plt.figure(figsize=(6,8))
            subplot = fig.add_subplot(1,k+1,1)
            #axis("off")
            plotidx = 2
            #Draw this image
            plt.imshow(image,cmap='Greys')
            break;
            k=k+1